In [1]:
from rope.base.project import Project

In [2]:
project = Project('../test-refactor/')

In [3]:
project.root.real_path

'/Users/deka.akbar/Projects/hackathon-auto-refactoring/test-refactor/'

In [4]:
from rope.contrib.generate import create_module, create_package

In [5]:
from rope.refactor import move

## Find the functions

In [6]:
from rope.refactor.occurrences import Finder

def find_function(src_res, function_name):
    finder = Finder(project, function_name)
    for occ in finder.find_occurrences(resource=src_res):
        return occ

In [7]:
src_res = project.get_resource("eg_nb.py")
dest_res = project.get_resource("mymod/file_to_be_moved.py")

In [8]:
occ = find_function(src_res, "myFuncToRefactor")

In [9]:
occ.lineno

3

In [10]:
occ.offset

53

In [11]:
src_res.read().index("myFuncToRefactor")

53

In [12]:
mover = move.create_move(project, src_res, occ.offset)

In [16]:
change_set = mover.get_changes(dest_res)

In [23]:
change_set.changes[0].get_changed_resources()

In [19]:
print(change_set.get_description())

Moving global <myFuncToRefactor>:


--- a/mymod/file_to_be_moved.py
+++ b/mymod/file_to_be_moved.py
@@ -0,0 +1,4 @@
+# Databricks notebook source
+# start of the file
+def myFuncToRefactor():
+    print("a")

--- a/eg_nb.py
+++ b/eg_nb.py
@@ -1,4 +0,0 @@
-# Databricks notebook source
-# start of the file
-def myFuncToRefactor():
-    print("a")




In [ ]:
# if the source is a notebook and the comment is gone, readd the "Databricks source file" at the top
# if the dest is not a notebook, remove the "Databricks source file" at the top
# if the dest is a notebook, check for the "Databricks source file" and add it at the top

In [34]:
src_res = project.get_resource("file_with_funcs.py")
# check if dest_res exist
# if it's a module create the nested modules first
dest_res = project.get_resource("mymod4/new/mod/new_move.py")

In [9]:
%%timeit -n 1 -r 1
find_function(src_res, "doesntexist")

2.44 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n 1 -r 1
find_function(src_res, "random_func")

1.26 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
%%timeit -n 1 -r 1
try:
    src_res.read().index("doesntexist")
except ValueError:
    pass

360 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%%timeit -n 1 -r 1
src_res.read().index("random_func")

514 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
occ = find_function(src_res, "random_func")

In [19]:
occ.offset

4

In [23]:
mover = move.create_move(project, src_res, occ.offset)

In [24]:
mover.old_name

'random_func'

In [35]:
changes = mover.get_changes(dest_res)

In [36]:
f1 = list(changes.get_changed_resources())[0]
f2 = list(changes.get_changed_resources())[1]

In [37]:
changes

In [38]:
changed_files = []
for file in changes.get_changed_resources():
    changed_files.append(file)
changed_files

In [44]:
changed_files[1].parent.path

'mymod4/new/mod'

In [39]:
# relative to the project path

In [40]:
changed_files[0].path

'file_with_funcs.py'

In [41]:
changed_files[1].path

'mymod4/new/mod/new_move.py'

In [48]:
changes.changes[0]

In [ ]:
project.do(changes)

## Moving

In [72]:
new_path = "mymod3/new/mod/newfile.py"

In [80]:
from rope.base import libutils
from pathlib import Path

In [ ]:
# algorithm

# - check if path is a file or nested file?
# - if it is a nested file --> create the module first for the nested folders if the folders don't exist
# - if it is a file --> create an empty module if the resource doesn't exist

In [130]:
def get_mod_from_path(path: Path):
    return str(path).strip("/").replace("/", ".")

In [168]:
from itertools import accumulate
import operator

In [176]:
for t in accumulate("a.b.c".split("."), func=lambda x, y: f"{x}.{y}"):
    print(t)

a
a.b
a.b.c


In [204]:
Path(new_path).suffix

'.py'

In [208]:
Path(new_path).with_suffix("")

PosixPath('mymod3/new/mod/newfile')

In [207]:
Path("mymod3/new/mod/newfile").suffix

''

In [228]:
from rope.base.exceptions import ResourceNotFoundError

In [234]:
project.get_module("mymod4")

In [229]:
def get_or_create_resource(path):
    base_path = project.root.real_path
    print(Path(base_path + path).exists())
    print(str(Path(base_path + path)))
    
    # if resource doesn't exist, create the module and/or parent subdirs first
    if not Path(base_path + path).exists():
        full_mod = ""
        # check if it contains subdirs first
        if Path(path).parent != ".":
            # create the subdir first using rope to create pkgs
            parent_mod = get_mod_from_path(Path(path).parent)
            for mod in accumulate(parent_mod.split("."), lambda x, y: f"{x}.{y}"):
                print(mod)
                try:
                    project.get_module(mod)
                except ModuleNotFoundError:
                    create_package(project, mod)
        
        # check if it is a module or a package
        # module --> .py, package is only name
        if Path(path).suffix == "":
            full_mod = get_mod_from_path(Path(path))
            create_package(project, full_mod)
        else:
            full_mod = get_mod_from_path(Path(path).with_suffix(""))
            create_module(project, full_mod)
        
    return project.get_resource(path)

In [235]:
get_or_create_resource("mymod4/new/mod/diff.py")

False
/Users/deka.akbar/Projects/hackathon-auto-refactoring/test-refactor/mymod4/new/mod/diff.py
mymod4
mymod4.new
mymod4.new.mod


In [233]:
get_or_create_resource("mymod4/new/mod/newfile")

False
/Users/deka.akbar/Projects/hackathon-auto-refactoring/test-refactor/mymod4/new/mod/newfile
mymod4
mymod4.new
mymod4.new.mod


In [200]:
pymod.get_resource().exist()

AttributeError: 'Folder' object has no attribute 'exist'

In [188]:
pymod.get_resource().exists()

True

In [164]:
get_or_create_resource(new_path)

ResourceNotFoundError: Resource <mymod3> does not exist

In [157]:
project.get_resource("new_file.py").get_resource()

AttributeError: 'File' object has no attribute 'get_resource'

In [109]:
project.root

In [121]:
new_path

'mymod3/new/mod/newfile.py'

In [125]:
Path(new_path).parent

PosixPath('mymod3/new/mod')

In [ ]:
if Path("newfile.py").

In [126]:
Path("newfile.py").parent

PosixPath('.')

In [127]:
Path("/newfile.py").parent

PosixPath('/')

In [112]:
Path(project.root.real_path + new_path).parent.mkdir(parents=True, exist_ok=True)

In [128]:
# create_package(project, "mymod3")

In [129]:
# create_module(project, "mymod3.mymod2")

In [113]:
project.get_module("mymod3.new")

In [107]:
create_package(project, new_path)

ResourceNotFoundError: Resource <mymod3/new/mod/newfile> does not exist

In [101]:
project.root.real_path

'/Users/deka.akbar/Projects/hackathon-auto-refactoring/test-refactor/'

In [102]:
# check for nested path's path

In [103]:
get_or_create_resource(new_path)

In [75]:
new_res = libutils.path_to_resource(project, new_path)
new_res

ResourceNotFoundError: Resource </Users/deka.akbar/Projects/hackathon-auto-refactoring/notebooks/mymod3/new/mod/newfile.py> does not exist

In [76]:
project.get_resource("mymod3/new/mod/newfile.py")

ResourceNotFoundError: Resource <mymod3/new/mod/newfile.py> does not exist